In [12]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,classification_report

# load the dataset
data = pd.read_csv("C:/Users/22r21/OneDrive/Documents/personal/OneDrive/Desktop/project1/Dyt-desktop.csv",index_col=0)
data = data.replace('No',0)
data = data.replace('Yes',1)


C:\Users\22r21\AppData\Local\Temp\ipykernel_11608\2929538066.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace('Yes',1)


In [13]:
# split the data into train and test sets
X = data.drop('Dyslexia', axis=1) 
y = data['Dyslexia'] # target variable
selector = SelectKBest(chi2, k=10)
X_new = selector.fit_transform(X, y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [14]:
y_pred=[1]*len(y_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC-ROC Score:", roc_auc_score(y_test, y_pred))
target_names = ['Does not have Dyslexia','Has Dyslexia']
print(classification_report(y_test, y_pred, target_names=target_names))


Accuracy: 0.10973936899862825
Precision: 0.10973936899862825
Recall: 1.0
F1 Score: 0.19777503090234858
AUC-ROC Score: 0.5
                        precision    recall  f1-score   support

Does not have Dyslexia       0.00      0.00      0.00       649
          Has Dyslexia       0.11      1.00      0.20        80

              accuracy                           0.11       729
             macro avg       0.05      0.50      0.10       729
          weighted avg       0.01      0.11      0.02       729



C:\Users\22r21\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\22r21\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\22r21\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [17]:
# Logistic Regression
logreg_default = LogisticRegression(max_iter=1000)
logreg_default.fit(X_train, y_train)
accuracy = logreg_default.score(X_test, y_test)
print("Logistic Regression default accuracy:",accuracy)

logistic_params = {"C": 10.0 ** np.arange(-10, 10)}
logistic_grid = GridSearchCV( LogisticRegression(max_iter=1000), logistic_params, cv=5)
logistic_grid.fit(X_train, y_train)
logistic_tuned_acc = accuracy_score(y_valid, logistic_grid.predict(X_valid))
logistic_params_best = logistic_grid.best_params_
print("logistic reg tuned validation accuracy: ",logistic_tuned_acc)
print(f"logistic selected hyperparameters: {logistic_params_best}")


logistic_best = LogisticRegression(max_iter=1000, C=logistic_params_best['C'])
logistic_best.fit(X_train, y_train)
accuracy_test = logistic_best.score(X_test, y_test)
print("Logistic Regression final test accuracy:",accuracy_test)


Logistic Regression default accuracy: 0.8902606310013718
logistic reg tuned validation accuracy:  0.902229845626072
logistic selected hyperparameters: {'C': 1e-10}
Logistic Regression final test accuracy: 0.8902606310013718


In [16]:
# k-Nearest Neighbors
knn_default = KNeighborsClassifier()
knn_default.fit(X_train, y_train)
accuracy = knn_default.score(X_test, y_test)
print("KNN default accuracy:",accuracy)


# k-NN
knn_params = {
    'n_neighbors': [5, 22, 10, 1, 15]
}
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5,scoring='accuracy')
knn_grid.fit(X_train, y_train)
knn_tuned_acc = accuracy_score(y_valid, knn_grid.predict(X_valid))
knn_params_best = knn_grid.best_params_
print("k-NN tuned validation accuracy: ",knn_tuned_acc)
print(f"k-NN selected hyperparameters: {knn_params_best}")

knn_best = KNeighborsClassifier(n_neighbors=knn_params_best['n_neighbors'])
knn_best.fit(X_train, y_train)
accuracy_test = knn_best.score(X_test, y_test)
print("KNN default accuracy:",accuracy_test)

KNN default accuracy: 0.8888888888888888
k-NN tuned validation accuracy:  0.902229845626072
k-NN selected hyperparameters: {'n_neighbors': 22}
KNN default accuracy: 0.8902606310013718
